In [6]:
products = [
    {
        "id": 1,
        "name": "Wireless Noise-Canceling Headphones",
        "description": "Immerse yourself in pure audio with these comfortable and powerful headphones. Perfect for travel and focused work.",
    },
    {
        "id": 2,
        "name": "Ergonomic Office Chair",
        "description": "Designed for all-day comfort and support, this chair helps improve posture and reduce back pain.",
    },
    {
        "id": 3,
        "name": "Smart Coffee Maker with Voice Control",
        "description": "Start your day right with a freshly brewed cup of coffee, controlled by your voice or a mobile app.",
    },
    {
        "id": 4,
        "name": "Lightweight Running Shoes",
        "description": "Engineered for speed and agility, these shoes provide excellent cushioning for your daily runs.",
    },
    {
        "id": 5,
        "name": "Bamboo Cutting Board Set",
        "description": "A durable and eco-friendly set of cutting boards for all your food preparation needs.",
    },
    {
        "id": 6,
        "name": "Portable Bluetooth Speaker",
        "description": "Enjoy your favorite music anywhere with this compact and water-resistant Bluetooth speaker.",
    },
]
# Example output

# query: "comfortable chair for working from home"
# output: "id": 2,
#     "name": "Ergonomic Office Chair",
#     "description": "Designed for all-day comfort and support, this chair helps improve posture and reduce back pain.",
#     "sim

In [1]:
from qdrant_client import QdrantClient

In [2]:
client = QdrantClient(":memory:")

In [3]:
from fastembed import TextEmbedding

In [4]:
embedding_model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 48545.19it/s]


In [7]:
products[0]

{'id': 1,
 'name': 'Wireless Noise-Canceling Headphones',
 'description': 'Immerse yourself in pure audio with these comfortable and powerful headphones. Perfect for travel and focused work.'}

In [18]:
text_embed = list(embedding_model.embed([products[0]["description"]]))[0]

In [19]:
text_embed.shape

(384,)

In [28]:
descriptions = []

for product in products:
    descriptions.append(product["description"])


In [29]:
descriptions

['Immerse yourself in pure audio with these comfortable and powerful headphones. Perfect for travel and focused work.',
 'Designed for all-day comfort and support, this chair helps improve posture and reduce back pain.',
 'Start your day right with a freshly brewed cup of coffee, controlled by your voice or a mobile app.',
 'Engineered for speed and agility, these shoes provide excellent cushioning for your daily runs.',
 'A durable and eco-friendly set of cutting boards for all your food preparation needs.',
 'Enjoy your favorite music anywhere with this compact and water-resistant Bluetooth speaker.']

In [40]:
descriptions_text_embeds = embedding_model.embed(descriptions)

In [42]:
embeddings_list = list(descriptions_text_embeds)


In [51]:
embeddings_list[0].shape

(384,)

In [49]:
qdrant_points = []
from qdrant_client import models

for product, embedding in zip(products, embeddings_list):
    qdrant_points.append(
        models.PointStruct(
            id=product["id"],
            vector=embedding,
            payload=product,
        )
    )

In [50]:
qdrant_points

[PointStruct(id=1, vector=[0.05060559883713722, -0.018527496606111526, 0.024595266208052635, -0.029715757817029953, 0.0007458498002961278, -0.020685790106654167, -0.00704601313918829, -0.017899636179208755, -0.012429042719304562, -0.015537302009761333, 0.009450222365558147, 0.004936115816235542, -0.029925070703029633, 0.021084534004330635, -0.003421572269871831, -0.0035635463427752256, -0.0007903326186351478, -0.04743140935897827, 0.04104623571038246, 0.04406103119254112, -0.017695745453238487, 0.0555838942527771, -0.006829369347542524, 0.031674761325120926, -0.00012267910642549396, 0.07638956606388092, 0.05259541794657707, -0.011097051203250885, 0.019326262176036835, -0.0874982699751854, -0.024285485967993736, -0.005271944683045149, -0.0026412492152303457, -0.026159800589084625, -0.010177964344620705, 0.010607577860355377, -0.001970189157873392, -0.026785913854837418, -0.008328273892402649, -0.02123359590768814, -0.05050278827548027, 0.007446140982210636, 0.026666251942515373, 0.02131

In [52]:
client.create_collection(
    collection_name="Shopping",
    vectors_config=models.VectorParams(
        size=embeddings_list[0].shape[0], distance=models.Distance.COSINE
    ),
)


True

In [53]:
client.upsert(
    collection_name="Shopping",
    points=qdrant_points,
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [65]:
query = "comfortable chair for working from home"
query_embed = list(embedding_model.embed([query]))

In [67]:
len(query_embed[0])

384

In [72]:
response = client.query_points(
    collection_name="Shopping",
    query=query_embed[0],
    limit=1,
)


In [73]:
response

QueryResponse(points=[ScoredPoint(id=2, version=0, score=0.855983720108078, payload={'id': 2, 'name': 'Ergonomic Office Chair', 'description': 'Designed for all-day comfort and support, this chair helps improve posture and reduce back pain.'}, vector=None, shard_key=None, order_value=None)])